## Imports

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import random
import unicodedata
import string
import torch
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from torch import nn, optim
from SearchClassifier.word_classifier.data_loader import random_training_example, split_dataset
from SearchClassifier.word_classifier.mutiLayerRNN import MultilayerRnn
from SearchClassifier.word_classifier.predict import evaluate, predict
from SearchClassifier.word_classifier.rnn import RNN
from SearchClassifier.word_classifier.test import test_with_plot
from SearchClassifier.word_classifier.train import train, train_loop
from SearchClassifier.word_classifier.utils import unicode_to_ascii, letter_to_tensor, line_to_tensor, n_letters, \
    category_from_output, load_checkpoint, save_checkpoint

## Global variables
Change for running configuration

In [2]:
TRAINING_PERCENT = 70
TEST_PERCENT = 20
VIEW_PERCENT = 10
LEARNING_RATE = 0.005  # If you set this too high, it might explode. If too low, it might not learn
EPOCHS = 5000
PRINT_EVERY = 100
PLOT_EVERY = 100
N_HIDDEN = 128
EXEC_MODE = 3
PATH2 = "models/model.pt"
PATH = "models/model2.pt"

### Configuration
Data loading and environment configuration

In [3]:
def find_files(path): return glob.glob(path)


print(find_files('data/products/*.txt'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []


# Read a file and split into lines
def read_lines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicode_to_ascii(line) for line in lines]


for filename in find_files('data/products/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = read_lines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

training_dataset, test_dataset, view_dataset = split_dataset(category_lines, TRAINING_PERCENT, TEST_PERCENT, VIEW_PERCENT)

line_to_tensor('Jose Ramón').size()

['data/products/toys.txt', 'data/products/music.txt', 'data/products/book.txt', 'data/products/smartphone.txt', 'data/products/clothes.txt', 'data/products/food.txt', 'data/products/games.txt', 'data/products/fitness.txt']


torch.Size([10, 1, 57])

## Creating RNN

In [4]:
if EXEC_MODE == 1:
    rnn = RNN(n_letters, N_HIDDEN, n_categories)
elif EXEC_MODE == 2:
    rnn = RNN(n_letters, N_HIDDEN, n_categories)
    rnn2 = MultilayerRnn(n_letters, N_HIDDEN, n_categories)
else:
    print("Loading network")
    rnn = MultilayerRnn(n_letters, N_HIDDEN, n_categories)
    rnn = load_checkpoint(rnn, PATH)
    #rnn.eval()
inputChar = letter_to_tensor('A')

hidden = torch.zeros(1, N_HIDDEN)

rnn

Loading network


MultilayerRnn(
  (i2h): Linear(in_features=185, out_features=128, bias=True)
  (h2h): Linear(in_features=128, out_features=128, bias=True)
  (i2o): Linear(in_features=185, out_features=8, bias=True)
  (softmax): LogSoftmax()
)

### Getting output

In [5]:
output, next_hidden = rnn(inputChar, hidden)
category_from_output(all_categories, output)

('toys', 0)

### Showing samples

In [6]:
for i in range(10):
    category, line, category_tensor, line_tensor = random_training_example(all_categories, training_dataset)
    print('category =', category, '/ line =', line)

category = smartphone / line = Xiaomi Mi MIX Alpha
category = smartphone / line = OnePlus T
category = games / line = Heavy Rain
category = music / line = The Predator
category = games / line = STAR WARS BATTLEFRONT II  Standard
category = clothes / line = Joma Nilo  Pantalones largos para hombre
category = music / line = Favourite Worst Nightmare Ed. Normal
category = music / line = Stevie Wonder The Definitive Collection 
category = music / line = Good Kid M.a.a.d City
category = games / line = Beyond Two Souls


### Defining criterion
NLLLoss() is good because the last layer of the RNN is nn.LogSoftmax I have to see the different criterion and activation functions

In [7]:
if EXEC_MODE == 1 or EXEC_MODE == 2:
    criterion = nn.NLLLoss()

## Training
each loop of training
  1) Create input and target tensors
  2) Create zeroed initial hidden state
  3) Read each letter in and keep hidden state for next letter
  4) Compare final output to target
  5) Back-propagate
  6) Return to output and loss
Keep track of losses for plotting

In [8]:
if EXEC_MODE == 1:
    print("Training network")
    rnn, all_losses = train_loop(rnn, criterion, all_categories, training_dataset, EPOCHS, LEARNING_RATE, PRINT_EVERY, PLOT_EVERY)
if EXEC_MODE == 2:
    print("Training first network")
    rnn, all_losses = train_loop(rnn, criterion, all_categories, training_dataset, EPOCHS, LEARNING_RATE, PRINT_EVERY, PLOT_EVERY)
    print("Training second network")
    rnn2, all_losses2 = train_loop(rnn2, criterion, all_categories, training_dataset, EPOCHS, LEARNING_RATE, PRINT_EVERY, PLOT_EVERY)

## Plotting info
1. One plot is a square of classes, in every square is represented the
index of times is predicted one class when it's the other
It's good to be yellow on diagonal
2. The other is de function loss progression

In [9]:
if EXEC_MODE == 1 or EXEC_MODE == 2:
    plt.figure()
    plt.plot(all_losses)
if EXEC_MODE == 2:
    plt.figure()
    plt.plot(all_losses2)

test_with_plot(rnn, test_dataset, all_categories)
if EXEC_MODE == 2:
    test_with_plot(rnn2, test_dataset, all_categories)

## Saving and Loading
Saving and Loading model functions

In [10]:
save_checkpoint(rnn, PATH)
if EXEC_MODE == 2:
    save_checkpoint(rnn2, PATH2)

### Some predicts

In [11]:
for key in view_dataset.keys():
    for value in view_dataset[key]:
        predict(rnn, value, all_categories, 3)


> Falomir Coloca  Juego de Mesa, Multicolor, unica 
tensor([[-1.7512, -1.8888, -1.8438, -2.4001, -2.8853, -4.1763, -1.4501, -2.1159]])
[[-1.7512081 -1.8888253 -1.8437657 -2.4000578 -2.885331  -4.1763034
  -1.4501361 -2.115911 ]]
-18.51153826713562
[array([0.0946009 , 0.10203503, 0.0996009 , 0.129652  , 0.15586662,
       0.22560543, 0.07833688, 0.11430229], dtype=float32)]
(0.23) food
(0.16) clothes
(0.13) smartphone

> LEGO Classic  Caja de ladrillos creativos grande, Set de Construccion con ladrillos de colores, Juguete Creativo y divertido a partir de  anos, incluye separador de piezas 
tensor([[-1.1139, -1.8237, -2.0653, -2.0869, -5.6983, -4.3263, -2.7347, -1.7261]])
[[-1.1139302 -1.8236889 -2.0652928 -2.0869036 -5.698251  -4.326251
  -2.7346869 -1.7261434]]
-21.57514750957489
[array([0.05163025, 0.08452729, 0.09572555, 0.0967272 , 0.2641118 ,
       0.20052011, 0.1267517 , 0.0800061 ], dtype=float32)]
(0.26) clothes
(0.20) food
(0.13) games

> Hasbro Gaming Don't Step In It Unorn